---

Here, our goal will be to create an AI that can learn to successfully play the classic snake game while attempting to obtain the highest score possible. Let's begin with importing our needed libraries.

In [1]:
from tqdm import tqdm
import numpy as np
import random
import pygame
import math

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Let's continue by setting the game parameters below.

In [2]:
# Let's initialize the imported Pygame modules #
pygame.init()

# Let's initialize color variables to color the snake, food, and window background #
green = (0,255,0)
black = (0,0,0)
red = (255,0,0)

# Let's set the display window values for the game #
dis_width = 500
dis_height = 500

# Let's set the size of the snake and the apple; as well as the speed of the snake #
snake_block = 10
snake_speed = 10
apple_block = snake_block

# Let's create a clock that will help us track time #
clock = pygame.time.Clock()

# Let's set how many games will be used to generate training data and the number of steps for each #
games_training = 1000
steps_training = 2000

# Let's set a seed for the random function #
random.seed(42)

Next, let's create some functions needed to run the game.

In [3]:
# Let's create a function that will draw our snake #
def snake(snake_pos, display):
    for position in snake_pos:
        pygame.draw.rect(display, green, pygame.Rect(position[0], position[1], snake_block, snake_block))

# Let's create a function that will draw our apple #
def apple(apple_pos, display):
    pygame.draw.rect(display, red, pygame.Rect(apple_pos[0], apple_pos[1], apple_block, apple_block))

# Let's create a function that will return the positions for the snake and apple; as well as the score #
def positions():
    snake_start = [(dis_width/2),(dis_height/2)]
    snake_pos = [snake_start]
    apple_pos = [round(random.randrange(0, dis_width - snake_block)/10.0)*10.0,
                 round(random.randrange(0, dis_height - snake_block)/10.0)*10.0]
    score = 0
    
    return snake_start, snake_pos, apple_pos, score

# Let's create a function that will return the distance between the apple and the snake #
def apple_dist_snake(apple_pos, snake_pos):
    return np.linalg.norm(np.array(apple_pos) - np.array(snake_pos[0]))

# Let's create a function that will allow the snake to move #
def move_snake(snake_start, snake_pos, apple_pos, button_dir, score):
    if button_dir == 1:
        snake_start[0] += snake_block
    elif button_dir == 0:
        snake_start[0] -= snake_block
    elif button_direction == 2:
        snake_start[1] += snake_block
    else:
        snake_start[1] -= snake_block
    
    # If-statement in the event that the apple is collected by the snake #
    if snake_start == apple_pos:
        apple_pos, score = collect_apple(apple_pos, score)
        snake_pos.insert(0, list(snake_start))
    else:
        snake_pos.insert(0, list(snake_start))
        snake_pos.pop()
    
    return snake_pos, apple_pos, score

# Let's create a function that will relocate the apple after collection #
def collect_apple(apple_pos, score):
    apple_pos = [round(random.randrange(0, dis_width - snake_block)/10.0)*10.0,
                 round(random.randrange(0, dis_height - snake_block)/10.0)*10.0]
    score += 1
    return apple_pos, score

# Let's create a function that will return if the snake collides with the display boundaries #
def boundary_collision(snake_start):
    if snake_start[0] >= dis_width or snake_start[0] < 0 or snake_start[1] >= dis_height or snake_start[1] < 0:
        return 1
    else:
        return 0

# Let's create a function that will return if the snake collides with itself #
def self_collision(snake_start, snake_pos):
    if snake_start in snake_position[1:]:
        return 1
    else:
        return 0

# Let's create a function that will allow the snake to look for blocked directions #
def blocked_dir(snake_pos):
    current_dir_vect = np.array(snake_pos[0]) - np.array(snake_pos[1])
    left_dir_vect = np.array([current_dir_vect[1], -current_dir_vect[0]])
    right_dir_vect = np.array([-current_dir_vect[1], current_dir_vect[0]])
    
    is_front_blocked = is_dir_blocked(snake_pos, current_dir_vect)
    is_left_blocked = is_dir_blocked(snake_pos, left_dir_vect)
    is_right_blocked = is_dir_blocked(snake_pos, right_dir_vect)
    
    return current_dir_vect, is_front_blocked, is_left_blocked, is_right_blocked

# Let's create a function that will return if the direction is blocked #
def is_dir_blocked(snake_pos, current_dir_vect):
    next_pos = snake_pos[0] + current_dir_vect
    snake_start = snake_pos[0]
    if boundary_collision(next_pos) == 1 or self_collision(next_pos.tolist(), snake_pos) == 1:
        return 1
    else:
        return 0

# Let's create a function that will generate a random direction for the snake given the angle with the apple #
def generate_rand_dir(snake_pos, apple_angle):
    direction = 0
    if apple_angle > 0:
        direction = 1
    elif apple_angle < 0:
        direction = -1
    else:
        direction = 0
    
    return direction_vector(snake_pos, apple_angle, direction)

# Let's create a function that will return the direction for the snake to move in #
def direction_vector(snake_pos, apple_angle, direction):
    current_dir_vect = np.array(snake_pos[0]) - np.array(snake_pos[1])
    left_dir_vect = np.array([current_dir_vect[1], -current_dir_vect[0]])
    right_dir_vect = np.array([-current_dir_vect[1], current_dir_vect[0]])
    
    new_direction = current_dir_vect
    
    if direction == -1:
        new_direction = left_dir_vect
    
    if direction == 1:
        new_direction = right_dir_vect
    
    button_dir = generate_button_dir(new_direction)
    
    return direction, button_dir

# Let's create a function that will generate the direction for the snake to move in #
def generate_button_dir(new_direction):
    button_dir = 0
    if new_direction.tolist() == [snake_block, 0]:
        button_dir = 1
    elif new_direction.tolist() == [-snake_block, 0]:
        button_dir = 0
    elif new_direction.tolist() == [0, snake_block]:
        button_dir = 2
    else:
        button_dir = 3
    
    return button_dir

# Let's create a function that will return the angle with the apple #
def apple_angle(snake_pos, apple_pos):
    apple_dir_vect = np.array(apple_pos) - np.array(snake_pos[0])
    snake_dir_vect = np.array(snake_pos[0]) - np.array(snake_pos[1])
    
    norm_apple_dir_vect = np.linalg.norm(apple_dir_vect)
    norm_snake_dir_vect = np.linalg.norm(snake_dir_vect)
    
    if norm_apple_dir_vect == 0:
        norm_apple_dir_vect = snake_block
    if norm_snake_dir_vect == 0:
        norm_snake_dir_vect = snake_block
    
    apple_dir_vect_normalized = apple_dir_vect / norm_apple_dir_vect
    snake_dir_vect_normalized = snake_dir_vect / norm_snake_dir_vect
    
    angle = math.atan2(
        apple_dir_vect_normalized[1] * snake_dir_vect_normalized[0] -
        apple_dir_vect_normalized[0] * snake_dir_vect_normalized[1],
        apple_dir_vect_normalized[1] * snake_dir_vect_normalized[1] +
        apple_dir_vect_normalized[0] * snake_dir_vect_normalized[0]) / math.pi
    
    return angle, snake_dir_vect, apple_dir_vect_normalized, snake_dir_vect_normalized

# Let's create a function that will run our game #
def run_game(snake_start, snake_pos, apple_pos, button_dir, score, display, clock):
    quit = False
    
    while quit is not True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit = True
        
        display.fill(black)
        
        apple(apple_pos, display)
        snake(snake_pos, display)
        
        snake_pos, apple_pos, score = move_snake(snake_start, snake_pos, apple_pos, button_dir, score)
        
        pygame.display.set_caption('SCORE: ' + str(score))
        
        pygame.display.update()
        
        clock.tick(snake_speed)
        
        return snake_pos, apple_pos, score

If we want our snake to undergo machine learning, we will need data for training. Let's create some functions to generate data to train our AI.

In [4]:
# Let's create a function to generate training data given a number of games and a number of steps for each #
def generate_training_data(display, clock):
    training_data_x = []
    training_data_y = []
    training_games = games_training
    steps_per_game = steps_training
    
    for _ in tqdm(range(training_games)):
        snake_start, snake_pos, apple_pos, score = positions()
        
        prev_apple_dist = apple_dist_snake(apple_pos, snake_pos)
        
        for _ in range(steps_per_game):
            angle, snake_dir_vect, apple_dir_vect_normalized, snake_dir_vect_normalized = apple_angle(snake_pos,
                                                                                                      apple_pos)
            direction, button_dir = generate_rand_dir(snake_pos, angle)
            
            current_dir_vect, is_front_blocked, is_left_blocked, is_right_blocked = blocked_dir(snake_pos)
            
            direction, button_dir, training_data_y = generate_training_data_y(snake_pos, apple_angle, button_dir,
                                                                              direction, training_data_y,
                                                                              is_front_blocked, is_left_blocked,
                                                                              is_right_blocked)
            
            if is_front_blocked == 1 and is_left_blocked == 1 and is_right_blocked == 1:
                break
            
            training_data_x.append([is_left_blocked, is_front_blocked, is_right_blocked,
                                    apple_dir_vect_normalized[0], snake_dir_vect_normalized[0],
                                    apple_dir_vect_normalized[1], snake_dir_vect_normalized[1]])
            
            snake_pos, apple_pos, score = run_game(snake_start, snake_pos, apple_pos,
                                                   button_dir, score, display, clock)
    
    return training_data_x, training_data_y

# Let's create a function to generate training data given every predicted direction #
def generate_training_data_y(snake_pos, apple_angle, button_dir, direction, training_data_y,
                             is_front_blocked, is_left_blocked, is_right_blocked):
    
    if direction == -1:
        
        if is_left_blocked == 1:
            
            if is_front_blocked == 1 and is_right_blocked == 0:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 1)
                
                training_data_y.append([0, 0, 1])
            
            elif is_front_blocked == 0 and is_right_blocked == 1:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 0)
                
                training_data_y.append([0, 1, 0])
            
            elif is_front_blocked == 0 and is_right_blocked == 0:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 1)
                
                training_data_y.append([0, 0, 1])
        
        else:
            training_data_y.append([1, 0, 0])
    
    elif direction == 0:
        
        if is_front_blocked == 1:
            
            if is_left_blocked == 1 and is_right_blocked == 0:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 1)
                
                training_data_y.append([0, 0, 1])
            
            elif is_left_blocked == 0 and is_right_blocked == 1:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, -1)
                
                training_data_y.append([1, 0, 0])
            
            elif is_left_blocked == 0 and is_right_blocked == 0:
                
                training_data_y.append([0, 0, 1])
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 1)
        
        else:
            
            training_data_y.append([0, 1, 0])
    
    else:
        
        if is_right_blocked == 1:
            
            if is_left_blocked == 1 and is_front_blocked == 0:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, 0)
                
                training_data_y.append([0, 1, 0])
            
            elif is_left_blocked == 0 and is_front_blocked == 1:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, -1)
                
                training_data_y.append([1, 0, 0])
            
            elif is_left_blocked == 0 and is_front_blocked == 0:
                
                direction, button_dir = direction_vector(snake_pos, apple_angle, -1)
                
                training_data_y.append([1, 0, 0])
        
        else:
            
            training_data_y.append([0, 0, 1])
    
    return direction, button_dir, training_data_y